<a href="https://colab.research.google.com/github/leonardussandy/CloudComparer/blob/master/DIPA_2024_Deteksi_Jalan_Raya_Berlubang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'road-damage-classification-and-assessment:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1215302%2F2029997%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240817%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240817T161959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D15ec2fbf6c4c45e9cde3190456e7532f8787f1de9ee06d5cc869de9e1f230f15d6d8933eb339f61765fa765a18b7fe4fa3a2f1cc7b1415c863921950d678e5a44b26b11d97fc744d74d95530e9a844bb00ea56b58d6253746807c478737e9f2d175e6270e40b22a454a29e7b3ddf716a9ec6553c7c48256dd636edc3792db34c89fb2495812f5bebd06a99ff7555c26d1e5866bf1cc4c6ca23c5b6276bfc9861d100cb5a56aee0a50d0d0fa6ce261423938961a13fc637f3957358ef15c1bf6c15339dfd301000a7e2b3a026dfeab3dc3ea5d1583ba33576b0d10db6463a0ba5b686f50950f41acec5765ca2eb995ea0f29a5bb0990224a8e0704be83be3252a,road-demage-4-class:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5113447%2F8556251%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240817%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240817T161959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D38a17b6fd76139d7a129dd38afe0d0fb95462ee9ffecbea290463d9032c9b532a42b86822e1446e50157b48de98fcec09e10d8288a9979b1a1270e2de429af1db570fcb9f1132c9a4d414aad677d791f8d4bb3a3dfccb14937ac837286094b7d7fc644170c35b5d47f415691b8e6cd59aa726caf4afb5cc74e7ca9ee47305f01296b404ce1ba38b9f81fc47c575b8bd52e17e6eff43fd63bad5153a31f1cda24ebd2d38926c637c28f8e168e8cb8015ba5cf15a8a28a99f08d3188c00d5b2e22900f51a326c58976df5bcba9fa2d6ea618273f60ad20c38566290ec9b66042546406c3df9827fa7032f1d67748f8a43789af9d06fb65ee6b31042699b3937b2e,4-class-road:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5113530%2F8556350%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240817%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240817T161959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D00cf70efaa23fc14b3c3284ec8ab556c81c00d603b9053693133f250aeb4a55c93a124a3f4c70bcc23b9679a2af360829322846912b14fbbe2309b5f79921180470a1ca38f4531a4ec5f53485db3394c68f12683154d5bb6333608b7169503fc55605a6b47398df2831ef53e5b7c4edfd725ec84751397ddfd18d72719b1272c2692b11df921ee0a407f4ab1bcf685bfbe00b298a699616563a9830afc339e017da026250f61c5573e5468c6b0a50666345bef3e59f4556d677bfd8cd701f87a2e2d90ccd6d5f002f06e186dc6725fd2541463a0fee5f5893094c3b6e8151ca8bf3905c8eefa9a6f7bcfcb5b368acbb06b68314fb78b1d3a5dd72ecd63123738,database-dipa-2024-jalan-berlubang:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5555961%2F9190782%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240817%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240817T161959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3a2c1af502f6042001fbe8b4b8a1a92cfd7b56537de1fdfb68a5f0031cf0b4fc93237cd4d997a871018767511afea6c530ae3082632f310131fe941a557dbade26b62dfa56aae7f862820765628a6e094b0d453859c2195b02134a93707ccb84be135cc9c8cb6fb9caa8a7186e63eb0e41c81fbc2dede152f69e1cf9bd55cd5808f6449c41b87120b8d542f641626aabb92c5f641c40fb5c3cc9d3b426ad7cd63be7a1b34bd0786ad4c9d8ffd00c60704bbfa89d1dcd9e6b6744c8ec20c8abfdd07dbc101ba6f796442e95eae0d5c73c71733b7d214bbe883b7372d460ac32ad0f746d2ccc02d92da26867eda0d8fb3a6ff5bb76dce1ae6ae610e117642e401b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[===============================                   ] 457850880 bytes downloaded

In [ ]:
!pip install ultralytics

# Importing the required libraries
from ultralytics import YOLO
import squarify
import matplotlib.pyplot as plt
import cv2
import os
import random
import pandas as pd
import matplotlib.image as mpimg
import seaborn as sns

sns.set_style('darkgrid')

%matplotlib inline
import ultralytics
ultralytics.checks()

In [ ]:
# Define the paths to the images and labels directories
train_images = "/kaggle/input/database-dipa-2024-jalan-berlubang/Database Jalan Berlubang/train/images"
train_labels = "/kaggle/input/database-dipa-2024-jalan-berlubang/Database Jalan Berlubang/train/labels"

test_images = "/kaggle/input/database-dipa-2024-jalan-berlubang/Database Jalan Berlubang/test/images"
test_labels = "/kaggle/input/database-dipa-2024-jalan-berlubang/Database Jalan Berlubang/test/labels"

val_images = "/kaggle/input/database-dipa-2024-jalan-berlubang/Database Jalan Berlubang/valid/images"
val_labels = "/kaggle/input/database-dipa-2024-jalan-berlubang/Database Jalan Berlubang/valid/labels"

# Get a list of all the image files in the training images directory
image_files = os.listdir(train_images)

# Choose 16 random image files from the list
random_images = random.sample(image_files, 16)

# Set up the plot
fig, axs = plt.subplots(4, 4, figsize=(16, 16))

# Loop over the random images and plot the object detections
for i, image_file in enumerate(random_images):
    row = i // 4
    col = i % 4

    # Load the image
    image_path = os.path.join(train_images, image_file)
    image = cv2.imread(image_path)

    # Load the labels for this image
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(train_labels, label_file)
    with open(label_path, "r") as f:
        labels = f.read().strip().split("\n")

    # Loop over the labels and plot the object detections
    # Loop over the labels and plot the object detections
    for label in labels:
        if len(label.split()) != 5:
            continue
        class_id, x_center, y_center, width, height = map(float, label.split())
        x_min = int((x_center - width/2) * image.shape[1])
        y_min = int((y_center - height/2) * image.shape[0])
        x_max = int((x_center + width/2) * image.shape[1])
        y_max = int((y_center + height/2) * image.shape[0])
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3)


    # Show the image with the object detections
    axs[row, col].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axs[row, col].axis('off')

plt.show()

In [ ]:
# Load an image using OpenCV
image = cv2.imread("/kaggle/input/database-dipa-2024-jalan-berlubang/Database Jalan Berlubang/train/images/pic-10-_jpg.rf.5a901c212d899a7dc7dc78be7de892c0.jpg")

# Get the size of the image
height, width, channels = image.shape
print(f"The image has dimensions {width}x{height} and {channels} channels.")

In [ ]:
# Loading a pretrained model
model = YOLO('yolov8x.pt')

# Training the model
model.train(data = '/kaggle/input/database-dipa-2024-jalan-berlubang/Database Jalan Berlubang/data.yaml',
            epochs = 100,
            imgsz = height,
            seed = 42,
            batch = 8,
            workers = 4)

In [ ]:
%matplotlib inline
# read in the results.csv file as a pandas dataframe
df = pd.read_csv('/kaggle/working/runs/detect/train/results.csv')
df.columns = df.columns.str.strip()

# create subplots using seaborn
fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(15, 15))

# plot the columns using seaborn
sns.lineplot(x='epoch', y='train/box_loss', data=df, ax=axs[0,0])
sns.lineplot(x='epoch', y='train/cls_loss', data=df, ax=axs[0,1])
sns.lineplot(x='epoch', y='train/dfl_loss', data=df, ax=axs[1,0])
sns.lineplot(x='epoch', y='metrics/precision(B)', data=df, ax=axs[1,1])
sns.lineplot(x='epoch', y='metrics/recall(B)', data=df, ax=axs[2,0])
sns.lineplot(x='epoch', y='metrics/mAP50(B)', data=df, ax=axs[2,1])
sns.lineplot(x='epoch', y='metrics/mAP50-95(B)', data=df, ax=axs[3,0])
sns.lineplot(x='epoch', y='val/box_loss', data=df, ax=axs[3,1])
sns.lineplot(x='epoch', y='val/cls_loss', data=df, ax=axs[4,0])
sns.lineplot(x='epoch', y='val/dfl_loss', data=df, ax=axs[4,1])

# set titles and axis labels for each subplot
axs[0,0].set(title='Train Box Loss')
axs[0,1].set(title='Train Class Loss')
axs[1,0].set(title='Train DFL Loss')
axs[1,1].set(title='Metrics Precision (B)')
axs[2,0].set(title='Metrics Recall (B)')
axs[2,1].set(title='Metrics mAP50 (B)')
axs[3,0].set(title='Metrics mAP50-95 (B)')
axs[3,1].set(title='Validation Box Loss')
axs[4,0].set(title='Validation Class Loss')
axs[4,1].set(title='Validation DFL Loss')

# add suptitle and subheader
plt.suptitle('Training Metrics and Loss', fontsize=24)

# adjust top margin to make space for suptitle
plt.subplots_adjust(top=0.8)

# adjust spacing between subplots
plt.tight_layout()

plt.show()

In [ ]:
%matplotlib inline
# Loading the best performing model
model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')


In [ ]:
# Evaluating the model on the test dataset
metrics = model.val(data="/kaggle/input/database-dipa-2024-jalan-berlubang/Database Jalan Berlubang/data.yaml")

In [ ]:
%matplotlib inline
# Create the barplot
ax = sns.barplot(x=['mAP50-95', 'mAP50', 'mAP75'], y=[metrics.box.map, metrics.box.map50, metrics.box.map75])

# Set the title and axis labels
ax.set_title('Evaluation Metrics')
ax.set_xlabel('Metric')
ax.set_ylabel('Value')

# Set the figure size
fig = plt.gcf()
fig.set_size_inches(8, 6)

# Add the values on top of the bars
for p in ax.patches:
    ax.annotate('{:.3f}'.format(p.get_height()), (p.get_x() + p.get_width() / 2, p.get_height()), ha='center', va='bottom')

# Show the plot
plt.show()


In [ ]:
%matplotlib inline
# Reading the confusion matrix image file
img = mpimg.imread('/kaggle/working/runs/detect/train/confusion_matrix.png')

# Plotting the confusion matrix image
fig, ax = plt.subplots(figsize = (15, 15))

ax.imshow(img)
ax.axis('off');

In [ ]:
# Function to perform ship detections
def ship_detect(img_path):

    # Read the image
    img = cv2.imread(img_path)

    # Pass the image through the detection model and get the result
    detect_result = model(img)

    # Plot the detections
    detect_img = detect_result[0].plot()

    # Convert the image to RGB format
    detect_img = cv2.cvtColor(detect_img, cv2.COLOR_BGR2RGB)

    return detect_img

In [ ]:
import random

# Define the directory where the custom images are stored
custom_image_dir = '/kaggle/input/database-dipa-2024-jalan-berlubang/Database Jalan Berlubang/test/images'

# Get the list of image files in the directory
image_files = os.listdir(custom_image_dir)

# Select 16 random images from the list
selected_images = random.sample(image_files, 16)

# Create a figure with subplots for each image
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

# Iterate over the selected images and plot each one
for i, img_file in enumerate(selected_images):

    # Compute the row and column index of the current subplot
    row_idx = i // 4
    col_idx = i % 4

    # Load the current image and run object detection
    img_path = os.path.join(custom_image_dir, img_file)
    detect_img = ship_detect(img_path)

    # Plot the current image on the appropriate subplot
    axes[row_idx, col_idx].imshow(detect_img)
    axes[row_idx, col_idx].axis('off')

# Adjust the spacing between the subplots
plt.subplots_adjust(wspace=0.05, hspace=0.05)

In [ ]:
# Function to perform ship detections
def ship_detect(img_path):

    # Read the image
    img = cv2.imread(img_path)

    # Pass the image through the detection model and get the result
    detect_result = model(img)

    # Plot the detections
    detect_img = detect_result[0].plot()

    # Convert the image to RGB format
    detect_img = cv2.cvtColor(detect_img, cv2.COLOR_BGR2RGB)

    return detect_img

In [ ]:
import random

# Define the directory where the custom images are stored
custom_image_dir = '/kaggle/input/database-dipa-2024-jalan-berlubang/Database Jalan Berlubang/test/images'

# Get the list of image files in the directory
image_files = os.listdir(custom_image_dir)

# Select 16 random images from the list
selected_images = random.sample(image_files, 16)

# Create a figure with subplots for each image
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

# Iterate over the selected images and plot each one
for i, img_file in enumerate(selected_images):

    # Compute the row and column index of the current subplot
    row_idx = i // 4
    col_idx = i % 4

    # Load the current image and run object detection
    img_path = os.path.join(custom_image_dir, img_file)
    detect_img = ship_detect(img_path)

    # Plot the current image on the appropriate subplot
    axes[row_idx, col_idx].imshow(detect_img)
    axes[row_idx, col_idx].axis('off')

# Adjust the spacing between the subplots
plt.subplots_adjust(wspace=0.05, hspace=0.05)